Step1: Mount the drive and install required packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install efficient_apriori
!pip install pyvis

Mounted at /content/drive


Step 2: Import packages

In [ ]:
import string
import pandas as pd
import numpy as np
import copy
import re
from efficient_apriori import apriori
from keras.preprocessing.text import text_to_word_sequence
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from pyvis.network import Network
import networkx as nx
!ln -s /usr/local/share/jupyter/nbextensions /nbextensions
%cd /nbextensions
!wget -q https://upload.wikimedia.org/wikipedia/commons/3/37/Youtube.svg

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/share/jupyter/nbextensions


Step 3: Create useful functions

In [ ]:
stop = stopwords.words('english')

class BaseTokenizer(object):
    def process_text(self, text):
        raise NotImplemented

    def process(self, texts):
        for text in texts:
            yield self.process_text(text)


RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],


    ' hitler ':
        [
            'hitler'
        ],

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' trans gender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k'
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],

    ' homo sex ual':
        [
            'homosexual'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                                                      'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],

    ' shit hole ':
        [
            'shythole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],


    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
}


class PatternTokenizer(BaseTokenizer):
    def __init__(self, lower=True, initial_filters=r"[^a-z0-9!@#\$%\^\&\*_\-,\.' ]", patterns=RE_PATTERNS,
                 remove_repetitions=True):
        self.lower = lower
        self.patterns = patterns
        self.initial_filters = initial_filters
        self.remove_repetitions = remove_repetitions

    def process_text(self, text):
        x = self._preprocess(text)
        for target, patterns in self.patterns.items():
            for pat in patterns:
                x = re.sub(pat, target, x)
        x = re.sub(r"[^a-z' ]", ' ', x)
        return x.split()

    def process_ds(self, ds):
        ### ds = Data series

        # lower
        ds = copy.deepcopy(ds)
        if self.lower:
            ds = ds.str.lower()
        # remove special chars
        if self.initial_filters is not None:
            ds = ds.str.replace(self.initial_filters, ' ')
        # fuuuuck => fuck
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL) 
            ds = ds.str.replace(pattern, r"\1")

        for target, patterns in self.patterns.items():
            for pat in patterns:
                ds = ds.str.replace(pat, target)

        ds = ds.str.replace(r"[^a-z' ]", ' ')

        return ds.str.split()

    def _preprocess(self, text):
        # lower
        if self.lower:
            text = text.lower()

        # remove special chars
        if self.initial_filters is not None:
            text = re.sub(self.initial_filters, ' ', text)

        # fuuuuck => fuck
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
            text = pattern.sub(r"\1", text)
        return text

Step 4: Read training data

In [ ]:
file_name = "/content/drive/MyDrive/Colab Notebooks/train.csv"
data = pd.read_csv(file_name)    
tokenizer = PatternTokenizer()
data["comment_text"] = tokenizer.process_ds(data["comment_text"]).str.join(sep=" ")

Step 5: Clean data

In [ ]:
def clean_text(text):
    text = re.sub('[{}]'.format(string.punctuation), ' ', text.lower())
    return ' '.join([word for word in text.split() if word not in (stop)])

data["comment_text"] = data["comment_text"].apply(clean_text)

Step 6: Generate association rules

In [ ]:
tags = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
from collections import defaultdict
records = []
for tag in tags[:1]:
    print(f"processing {tag}")
    train = data["comment_text"][data[tag] == 1]
    records.extend([text.split() for text in train])
itemsets, rules = apriori(records, min_support=0.005, min_confidence=0.7, verbosity=1)
result = defaultdict(set)
for rule in rules:
    left, right = rule.lhs, rule.rhs
    for left_side in left:
        for right_side in right:
            result[left_side].add(right_side)
    
print(result)

processing toxic
Generating itemsets.
 Counting itemsets of length 1.
  Found 30828 candidate itemsets of length 1.
  Found 608 large itemsets of length 1.
 Counting itemsets of length 2.
  Found 184528 candidate itemsets of length 2.
  Found 1363 large itemsets of length 2.
 Counting itemsets of length 3.
  Found 7461 candidate itemsets of length 3.
  Found 262 large itemsets of length 3.
 Counting itemsets of length 4.
  Found 140 candidate itemsets of length 4.
  Found 4 large itemsets of length 4.
 Counting itemsets of length 5.
  Found 0 candidate itemsets of length 5.
Itemset generation terminated.

Generating rules from itemsets.
 Generating rules of size 2.
 Generating rules of size 3.
 Generating rules of size 4.
Rule generation terminated.

defaultdict(<class 'set'>, {'ahead': {'go'}, 'alone': {'leave'}, 'hole': {'ass', 'fuck'}, 'attacks': {'personal'}, 'bull': {'shit'}, 'unsigned': {'comment'}, 'ed': {'retard'}, 'mother': {'fuck'}, 'ual': {'homo', 'sex'}, 'org': {'wikipedia'

Step 7: Gnerate output for creating dynamic html page

In [ ]:
edges = set()
colors = [
"Yellow",
"Blue",
"Red",
"Green",
"Black",
"Orange",
"Brown",
"Ivory",
"Teal",
"Silver",
"Purple",
"Navy blue",
"Pea green",
"Gray",
"Maroon",
"Charcoal",
"Aquamarine",
"Coral",
"Fuchsia",
"Wheat",
"Lime",
"Crimson",
"Khaki",
"Hot pink",
"Magenta",
"Olden",
"Plum",
"Olive",
"Cyan"]
nodes = defaultdict(int)
for key, value in result.items():
    nodes[key] += len(value)
    for second in value:
        nodes[second] += 1
    for second in value:
        edges.add(tuple([key, second]))
count = 0
labels = dict()
for key, val in nodes.items():
    labels[key] = count
    color = colors[count % len(colors)]
    print(f'pyvis_graph.add_node({count}, label="{key}", color="{color}", size={val * 5})')
    count += 1

count = 10
for u, v in edges:
    u = labels[u]
    v = labels[v]
    color = colors[count % len(colors)]
    count += 1
    print(f'pyvis_graph.add_edge({u}, {v}, size=10, color="{color}")')


pyvis_graph.add_node(0, label="ahead", color="Yellow", size=5)
pyvis_graph.add_node(1, label="go", color="Blue", size=15)
pyvis_graph.add_node(2, label="alone", color="Red", size=5)
pyvis_graph.add_node(3, label="leave", color="Green", size=5)
pyvis_graph.add_node(4, label="hole", color="Black", size=10)
pyvis_graph.add_node(5, label="ass", color="Orange", size=45)
pyvis_graph.add_node(6, label="fuck", color="Brown", size=55)
pyvis_graph.add_node(7, label="attacks", color="Ivory", size=5)
pyvis_graph.add_node(8, label="personal", color="Teal", size=5)
pyvis_graph.add_node(9, label="bull", color="Silver", size=5)
pyvis_graph.add_node(10, label="shit", color="Purple", size=30)
pyvis_graph.add_node(11, label="unsigned", color="Navy blue", size=5)
pyvis_graph.add_node(12, label="comment", color="Pea green", size=5)
pyvis_graph.add_node(13, label="ed", color="Gray", size=5)
pyvis_graph.add_node(14, label="retard", color="Maroon", size=5)
pyvis_graph.add_node(15, label="mother", color="Charc

Step 8: Run `association_rule_generator.py` to get dynamic html page